In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test.csv
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_train.csv
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-221.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-29.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-453.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-70.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-312.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-646.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-28.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-483.jpg
/kaggle/input/mental-health-meme-classification/Anxiety_Data/anxiety_test_image/TE-66.jpg
/kaggle/input/mental-health-

In [ ]:
import os
import json
import pandas as pd
from PIL import Image
import pytesseract
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, f1_score

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# Dataset for Depression Data
# -------------------------------
# Assumption: The JSON file (train.json) has two keys:
# "sample_id" (the image file name without extension) and
# "meme_depressive_categories" (the corresponding depression symptom labels as a list).
class DepressionDataset(Dataset):
    def __init__(self, json_file, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform

        # Load JSON file.
        with open(json_file, 'r') as f:
            data = json.load(f)
        # Convert list of dictionaries to DataFrame.
        data = pd.DataFrame(data)
        print("JSON keys:", data.columns)

        # Debug: print first few sample_ids.
        print("First few sample_ids from JSON:")
        print(data['sample_id'].head())

        # Append ".jpeg" to sample_id if not already present.
        data['filename'] = data['sample_id'].apply(lambda x: str(x) + ".jpeg" if not str(x).endswith(".jpeg") else x)

        # Convert list labels to string (join with a delimiter) if necessary.
        data['meme_depressive_categories'] = data['meme_depressive_categories'].apply(
            lambda x: '|'.join(x) if isinstance(x, list) and len(x) > 0 else str(x)
        )

        # Convert string labels to integer labels using factorize.
        data['label'] = pd.factorize(data['meme_depressive_categories'])[0]
        print("Unique label mapping:", dict(enumerate(pd.factorize(data['meme_depressive_categories'])[1])))

        # Check existence for each row and filter out missing ones.
        data['exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(self.image_folder, x)))
        print("Number of existing files found:", data['exists'].sum())

        self.data = data[data['exists']].reset_index(drop=True)
        print(f"Filtered dataset length: {len(self.data)} (only rows with existing images)")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_name = row['filename']
        label = row['label']
        image_path = os.path.join(self.image_folder, image_name)

        # Open image and convert to RGB.
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Extract OCR text using pytesseract.
        ocr_text = pytesseract.image_to_string(Image.open(image_path).convert("RGB"))
        return ocr_text, label

# -------------------------------
# Module 1: Figurative Reasoning
# -------------------------------
# Placeholder module: In a complete system, replace this with prompt engineering (e.g., using GPT-4).
class FigurativeReasoningModule(nn.Module):
    def __init__(self):
        super(FigurativeReasoningModule, self).__init__()

    def forward(self, ocr_text):
        reasoning = "dummy figurative reasoning for: " + ocr_text
        return reasoning

# -------------------------------
# RAG Module
# -------------------------------
class RAGModule(nn.Module):
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
        super(RAGModule, self).__init__()
        # Load SentenceTransformer on CPU.
        self.encoder = SentenceTransformer(model_name, device="cpu")

    def encode_text(self, text):
        embedding = self.encoder.encode(text, convert_to_tensor=True)
        return embedding

    def fuse_embeddings(self, ocr_text, reasoning):
        emb_ocr = self.encode_text(ocr_text)
        emb_reasoning = self.encode_text(reasoning)
        fused = torch.cat([emb_ocr, emb_reasoning], dim=-1)
        # Move the fused embedding to the target device.
        return fused.to(device)

    def retrieve_similar(self, query_embedding, database_embeddings, top_k=1):
        cos_scores = util.cos_sim(query_embedding, database_embeddings)
        top_results = torch.topk(cos_scores, k=top_k)
        return top_results

# -------------------------------
# Classifier Module
# -------------------------------
class M3HClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(M3HClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, fused_embedding):
        logits = self.fc(fused_embedding)
        return logits

# -------------------------------
# Full M3H Model: Integration of RAG and Classifier
# -------------------------------
class M3HModel(nn.Module):
    def __init__(self, rag_module, classifier):
        super(M3HModel, self).__init__()
        self.rag_module = rag_module
        self.classifier = classifier

    def forward(self, ocr_text, reasoning):
        fused_embedding = self.rag_module.fuse_embeddings(ocr_text, reasoning)
        if fused_embedding.dim() == 1:
            fused_embedding = fused_embedding.unsqueeze(0)
        logits = self.classifier(fused_embedding)
        return logits

# -------------------------------
# Training and Evaluation for Depression Dataset
# -------------------------------
if __name__ == "__main__":
    # File paths for the depression dataset.
    json_train = '/kaggle/input/mental-health-meme-classification/Depressive_Data/Depressive_Data/train.json'
    image_folder = '/kaggle/input/mental-health-meme-classification/Depressive_Data/Depressive_Data/Images/depressive_image/train'

    # Instantiate modules.
    rag_module = RAGModule()
    figurative_module = FigurativeReasoningModule()
    fused_dim = 768  # For paraphrase-MiniLM-L6-v2: 384*2 = 768
    num_classes = 16  # Adjust based on the depression dataset unique labels.
    classifier = M3HClassifier(input_dim=fused_dim, num_classes=num_classes)
    model = M3HModel(rag_module, classifier)
    model.to(device)

    # Optimizer and loss.
    optimizer = optim.Adam(model.parameters(), lr=5e-5)
    criterion = nn.CrossEntropyLoss()

    # Define image transformations.
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # Create the DepressionDataset and DataLoader.
    dataset_dep = DepressionDataset(json_file=json_train, image_folder=image_folder, transform=transform)
    dataloader_dep = DataLoader(dataset_dep, batch_size=32, shuffle=True)

    # Training loop for multiple epochs.
    num_epochs = 3
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in dataloader_dep:
            ocr_texts, labels = batch
            reasoning_batch = [figurative_module(text) for text in ocr_texts]
            logits = model(ocr_texts, reasoning_batch)
            labels = torch.tensor(labels, dtype=torch.long).to(device)
            loss = criterion(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            print(f"Epoch {epoch+1} Batch loss: {loss.item()}")
        avg_loss = total_loss / len(dataloader_dep)
        print(f"Epoch {epoch+1} Average loss: {avg_loss}")


Using device: cuda
JSON keys: Index(['sample_id', 'ocr_text', 'meme_depressive_categories'], dtype='object')
First few sample_ids from JSON:
0    TR-1
1    TR-2
2    TR-3
3    TR-4
4    TR-5
Name: sample_id, dtype: object
Unique label mapping: {0: 'Eating Disorder', 1: 'Self-Harm', 2: 'Feeling Down', 3: 'Low Self-Esteem', 4: 'Feeling Down|Low Self-Esteem', 5: 'Self-Harm|Feeling Down|Low Self-Esteem', 6: 'Feeling Down|Self-Harm', 7: 'Self-Harm|Low Self-Esteem', 8: 'Low Self-Esteem|Feeling Down|Lack of Energy', 9: 'Feeling Down|Lack of Energy', 10: 'Low Self-Esteem|Lack of Energy', 11: 'Lack of Energy', 12: 'Self-Harm|Feeling Down|Lack of Energy', 13: 'Sleeping Disorder', 14: 'Concentration Problem', 15: 'Lack of Interest'}
Number of existing files found: 8814
Filtered dataset length: 8814 (only rows with existing images)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-8-d2d21b53a306>:182: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 1 Batch loss: 2.809283971786499


...

Epoch 3 Batch loss: 1.7658627033233643


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 3 Batch loss: 1.8428579568862915
Epoch 3 Average loss: 1.928834951442221


<ipython-input-8-d2d21b53a306>:182: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


In [ ]:
# Evaluation.
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in dataloader_dep:
        ocr_texts, labels = batch
        reasoning_batch = [figurative_module(text) for text in ocr_texts]
        logits = model(ocr_texts, reasoning_batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(torch.tensor(labels).cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')

print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Macro-F1:", macro_f1)
print("Weighted-F1:", weighted_f1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-11-07694a9296cf>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_labels.extend(torch.tensor(labels).cpu().numpy())


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

<ipython-input-11-07694a9296cf>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_labels.extend(torch.tensor(labels).cpu().numpy())


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluation Metrics:
Accuracy: 0.4014068527342864
Macro-F1: 0.11085986688120217
Weighted-F1: 0.3403920871232268


<ipython-input-11-07694a9296cf>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_labels.extend(torch.tensor(labels).cpu().numpy())
